In [1]:
# Importing dependencies
import pandas as pd
import pymongo

In [2]:
# Reading in .csv's
election_df = pd.read_csv("Data/1976-2016-president.csv")

In [3]:
# Initializing a DataFrame with 2016 presidential election data relevant to our project
votes_per_state_2016_df = election_df.loc[election_df["year"] == 2016, ["state", "year", "party",
                                                                        "candidatevotes", "totalvotes"]]

In [4]:
# Displaying votes_per_state_2016_df
votes_per_state_2016_df.head()

,state,year,party,candidatevotes,totalvotes
3395,Alabama,2016,republican,1318255,2123372
3396,Alabama,2016,democrat,729547,2123372
3397,Alabama,2016,libertarian,44467,2123372
3398,Alabama,2016,NaN,21712,2123372
3399,Alabama,2016,green,9391,2123372


In [5]:
# Filling the blank spaces in the party name column for write-in candidates with "write-in"
votes_per_state_2016_df = votes_per_state_2016_df.fillna("write-it")
# Displaying votes_per_state_2016_df
votes_per_state_2016_df.head()

,state,year,party,candidatevotes,totalvotes
3395,Alabama,2016,republican,1318255,2123372
3396,Alabama,2016,democrat,729547,2123372
3397,Alabama,2016,libertarian,44467,2123372
3398,Alabama,2016,write-it,21712,2123372
3399,Alabama,2016,green,9391,2123372


In [6]:
# Initializing a new column in votes_per_state_2016_df to hold the percentage of total votes each party's votes 
# make up
votes_per_state_2016_df["percentage of total votes"] = (votes_per_state_2016_df["candidatevotes"] 
                                                     / votes_per_state_2016_df["totalvotes"])*100

In [7]:
# Displaying votes_per_state_2016_df
votes_per_state_2016_df.head()

,state,year,party,candidatevotes,totalvotes,percentage of total votes
3395,Alabama,2016,republican,1318255,2123372,62.083092
3396,Alabama,2016,democrat,729547,2123372,34.357946
3397,Alabama,2016,libertarian,44467,2123372,2.094169
3398,Alabama,2016,write-it,21712,2123372,1.022525
3399,Alabama,2016,green,9391,2123372,0.442268


In [8]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
# Define database and collection
db = client.etl_db
collection = db.state_votes
db.state_votes.drop()

In [10]:
# Initializing a variable with the names of each state in the votes_per_state_2016_df
state_list = votes_per_state_2016_df["state"].unique()

In [11]:
# Using a for-loop to loop through votes_per_state_2016_df in order to update post with relevant voting data for each 
# state, and then to create a new MongoDB document for each state in state_voting_db
for state in state_list:

    # Initializing variables with a list of each state's political parties and total votes
    parties = votes_per_state_2016_df.loc[votes_per_state_2016_df["state"] == state, "party"].to_list()
    total_votes = votes_per_state_2016_df.loc[votes_per_state_2016_df["state"] == state, "totalvotes"].to_list()[0]
    # Initializing post with state name, year, and total_votes
    post = {
                "state": state,
                "year": 2016,
                "total_votes": total_votes
            } 
    # Using a for-loop to loop throught the states in state_list and retrieve the remaining voting data
    for party in parties:
        # Initializing a variable with the number of votes for the party in question
        party_votes = votes_per_state_2016_df.loc[(votes_per_state_2016_df["state"] == state)
                                               & (votes_per_state_2016_df["party"] == party), 
                                                  "candidatevotes"].to_list()[0]
       
        # Initializing a variable with the percentage of the party in question's votes of the state's total votes
        percentage_of_total_votes = votes_per_state_2016_df.loc[(votes_per_state_2016_df["state"] == state)
                                                            & (votes_per_state_2016_df["party"] == party), 
                                                            "percentage of total votes"].to_list()[0]

        # Using a conditional to remove any potential "."'s from party names
        if "." in party:
            party_formatted = party.replace(".", "")
            party_stats = {f"{party_formatted}_votes": party_votes, 
                           f"{party_formatted}_percentage": percentage_of_total_votes}
            post[f"{party_formatted}"] = party_stats
        elif "." not in party:
            party_stats = {f"{party}_votes": party_votes, f"{party}_percentage": percentage_of_total_votes}
            post[f"{party}"] = party_stats
    # Storing the state's voting data as a MongoDB document in state_voting_db
    collection.insert_one(post)
    